In [1]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random
import threading
import coinconfig

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.helpers import round_step_size
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [2]:
#0:BTCBUSD, 1:BTCUSDT, 2:DOGEUSDT, 3:DOGEBUSD, 4:ETHBUSD, 5:ETHUSDT, 6:EOSUSDT, 7:LTCUSDT, 8:BNBBUSD

In [3]:
client = Client(KEY, SECRET, testnet=False)
config=coinconfig.tradeconfig
coin=coinconfig.coinlist[4]

In [4]:
symbol=coin["symbol"]
asset=coin["asset"]
recvWindow=config["recvWindow"]
dualSidePosition=config["dualSidePosition"]
marginType=config["marginType"]

leverage=coin["leverage"]
roundity=coin["roundity"]
profit_percent=coin["profit_percent"]
risk=coin["risk"]
stop_percent=profit_percent-risk
balance_part=coin["balance_part"]
compensation=coin["compensation"]
price_divergence=coin['price_divergence']

In [5]:
client.futures_change_leverage(symbol=symbol, leverage=leverage, recvWindow=recvWindow)

{'symbol': 'ETHBUSD', 'leverage': 50, 'maxNotionalValue': '50000'}

In [6]:
# Margin type
def margin_type_to_isolated():
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    body=positions[positions['symbol']==symbol]['isolated'].astype(bool).tolist()[0]

    if body==True:
        client.futures_change_margin_type(symbol=symbol, marginType=marginType, recvWindow=recvWindow)
        print(f'Changed to {marginType}')
    else:
        print(f'Already {marginType}')

In [7]:
margin_type_to_isolated()

Already CROSSED


In [8]:
# Hedge mode turn on

def hedge_mode_turn_on():
    status=client.futures_get_position_mode() 
    if status['dualSidePosition']==False:
        client.futures_change_position_mode(dualSidePosition=dualSidePosition, recvWindow=recvWindow)
        client.futures_get_position_mode()
        print('Dual turned on')
    else:
        print('Already Dual')
    return(status['dualSidePosition'])

In [9]:
hedge_mode_turn_on()

Already Dual


True

In [10]:
def get_assetbalance(symbol):
    status = client.futures_account()
    balance = pd.DataFrame(status['assets'])
    assetbalance = float(balance.loc[balance['asset'] == asset, 'availableBalance'])
    return(assetbalance)

In [11]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    long_position=float(open_positions[open_positions['positionSide']=='LONG']['entryPrice'])
    short_position=float(open_positions[open_positions['positionSide']=='SHORT']['entryPrice'])

    return([long_position,short_position])

In [12]:
def get_quant(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    quant=float(open_positions[open_positions['positionSide']=='LONG']['positionAmt'])
    if quant==0:
        quant=abs(float(open_positions[open_positions['positionSide']=='SHORT']['positionAmt']))
    return(quant)

In [13]:
quant=get_quant(symbol)

In [14]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [15]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [16]:
get_symbol_price(symbol)

1266.56

In [17]:
#получить tick size

def get_tick_size(symbol):
    data = client.futures_exchange_info()
    tick_size = 0.0
    the_symbol = symbol

    found = False
    info = data['symbols']
    for s in range(len(info)):
        if info[s]['symbol'] == the_symbol:
            filters = info[s]['filters']
            for f in range(len(filters)):
                if filters[f]['filterType'] == 'PRICE_FILTER':
                    tick_size = float(filters[f]['tickSize'])
                    found = True
                    break
            break
    return(tick_size)

In [18]:
#Trade config

# maxposition=0.016 #1 = 100% = 1 BTC,
maxposition=round(get_assetbalance(symbol)*balance_part*leverage/get_symbol_price(symbol),3)
quantity=str(round(maxposition/2,roundity))
tick_size=get_tick_size(symbol)
symbolround=abs(str(tick_size).find('.') - len(str(tick_size))) - 1
quantity

'0.017'

In [19]:
#Stop loss LONG 
def open_sl_long(symbol,quant,tick_size):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        stop_price_trigger=str(round(float(entry_price)*(1-stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }

        open_position_sl_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_sl_opened=float(open_position_sl_long_response[0]['price'])
        print('LONG sl order opened price:'+str(long_sl_opened))
        return(long_sl_opened)
    
    except :
        print('\n\nSL long not open...')
        

In [20]:
#Take Profit LONG 

def open_tp_long(symbol,quant,tick_size):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        take_price_trigger=str(round(float(entry_price)*(1+profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }
        open_position_tp_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_tp_opened=float(open_position_tp_long_response[0]['price'])
        print('LONG tp order opened price:'+str(long_tp_opened))
        return(long_tp_opened)
        
    except :
        print('\n\nTP long not open...')

In [21]:
#Открыть sl&tp LONG если есть позиция

def long_tp_sl_opening(symbol,quantity):
    position=get_opened_positions(symbol)
    if position[0]>0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            x = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size))
            x.start()
            y = threading.Thread(target=open_sl_long, args=(symbol,quantity,tick_size))
            y.start()
        else:
            order_long=orders_df[orders_df['positionSide']=='LONG']
            long_tp=order_long[order_long['origType']=='TAKE_PROFIT']['price']
            long_sl=order_long[order_long['origType']=='STOP']['price']
            if (long_tp+long_sl).empty:
                x = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size))
                x.start()
                y = threading.Thread(target=open_sl_long, args=(symbol,quantity,tick_size))
                y.start()
            else:
                print('LONG SL TP iteration continious..')

In [22]:
#Открыть позицию LONG 

def open_position_long(symbol,quantity,tick_size):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1+0.001),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "BUY",
                "positionSide": "LONG",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    long_opened=float(open_position_long_response[0]['price'])
    print('LONG order opened price:'+str(long_opened))
    if long_opened>0:
        long_tp_sl_opening(symbol,quantity)
    return(long_opened)

In [23]:
#Stop loss SHORT 

def open_sl_short(symbol,quant,tick_size):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        stop_price_trigger=str(round(float(entry_price)*(1+stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }
        
        open_position_sl_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_sl_opened=float(open_position_sl_short_response[0]['price'])
        print('SHORT sl order opened price:'+str(short_sl_opened))
        return(short_sl_opened)
    
    except :
        print('\n\nSL short not open...')


In [24]:
#Take Profit SHORT

def open_tp_short(symbol,quant,tick_size):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        take_price_trigger=str(round(float(entry_price)*(1-profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }

        open_position_tp_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_tp_opened=float(open_position_tp_short_response[0]['price'])
        print('SHORT tp order opened price:'+str(short_tp_opened))
        return(short_tp_opened)
    
    except :
        print('\n\nTp short not open...')


In [25]:
#Открыть sl&tp SHORT если есть позиция
       
def short_tp_sl_opening(symbol,quantity):
    position=get_opened_positions(symbol)
    if position[1]>0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if orders_df.empty:
            x = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size))
            x.start()
            y = threading.Thread(target=open_sl_short, args=(symbol,quantity,tick_size))
            y.start()
        else:
            order_short=orders_df[orders_df['positionSide']=='SHORT']
            short_tp=order_short[order_short['origType']=='TAKE_PROFIT']['price']
            short_sl=order_short[order_short['origType']=='STOP']['price']
            if (short_tp+short_sl).empty:
                x = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size))
                x.start()
                y = threading.Thread(target=open_sl_short, args=(symbol,quantity,tick_size))
                y.start()
            else:
                print('SHORT SL TP iteration continious..')

In [26]:
#Открыть позицию SHORT

def open_position_short(symbol,quantity,tick_size):
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1-0.001),symbolround))
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "SELL",
                "positionSide": "SHORT",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    short_opened=float(open_position_short_response[0]['price'])
    print('SHORT order opened price:'+str(short_opened))
    if short_opened>0:
        short_tp_sl_opening(symbol,quantity)
    return(short_opened)

In [27]:
def check_positions_ready(symbol):
    positions=get_opened_positions(symbol)
    if positions[0] and positions[1] > 0:
        orders=client.futures_get_open_orders(symbol=symbol)
        orders_df=pd.DataFrame(orders)
        if len(orders_df.index)==2:
            quant=str(get_quant(symbol))
            if orders_df['positionSide'][0]=='SHORT':
                long_tp_sl_opening(symbol,quant)
            else:
                short_tp_sl_opening(symbol,quant)
        else:
            print('TP/SL stage ок')

In [28]:
def check_avg(symbol):
    x = requests.get('https://binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(50)+'&interval=1m')
    dataset=pd.DataFrame(x.json())
    dataset.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    dataset['open']=dataset['open'].astype(float)
    openmean=dataset['open'].mean()
    check=abs(openmean-get_symbol_price(symbol))/openmean
    print(check)
    verdict=1
    if check < price_divergence:
        verdict=0
    return(verdict)

In [29]:
def check_positions_new(symbol):
    positions=get_opened_positions(symbol)
    avg=check_avg(symbol)
    if avg==1:
        if positions[0]+positions[1]==0:
            x = threading.Thread(target=open_position_short, args=(symbol,quantity,tick_size))
            x.start()
            y = threading.Thread(target=open_position_long, args=(symbol,quantity,tick_size))
            y.start()
            print('Start new positions')
    else:
        print('Near average price')

In [30]:
def check_one_stop(symbol):
    orders=client.futures_get_open_orders(symbol=symbol)
    df=pd.DataFrame(orders)
    if len(df.index)==2:
        quant=str(get_quant(symbol))
        positions=get_opened_positions(symbol)
        price=get_symbol_price(symbol)
        if positions[0]==0: #если остался шорт, то цена закрытия не должна быть выше чем (цена входа *(1- процент компенсации) 
            compensation_price=positions[1]*(1-(stop_percent*compensation))
            print('SHORT compansation stage')
            if price > compensation_price: #если шорт отрос на 0.4
                params={ 
                    "batchOrders": [
                        {
                            "symbol":symbol, 
                            "positionSide": "SHORT", 
                            "side":'BUY', 
                            "type":'MARKET',
                            "quantity":quant
                        }
                    ]
                }
                one_stop=send_signed_request('POST', '/fapi/v1/batchOrders', params)
                if str(float(one_stop[0]['origQty']))==quant:
                    print(f'Compensation SHORT price {compensation_price}')
            else:
                print('One step for SHORT not in compensation condition')
        if positions[1]==0:
            compensation_price=positions[0]*(1+(stop_percent*compensation))
            print('LONG compansation stage')
            if price < compensation_price: #если лонг упал на 0.4
                params={ 
                    "batchOrders": [
                        {
                            "symbol":symbol, 
                            "positionSide": "LONG", 
                            "side":'SELL', 
                            "type":'MARKET',
                            "quantity":quant
                        }
                    ]
                }
                one_stop=send_signed_request('POST', '/fapi/v1/batchOrders', params)
                if str(float(one_stop[0]['origQty']))==quant:
                    print(f'Compensation LONG price {compensation_price}')
            else:
                print('One step for LONG not in compensation condition')


In [31]:
def main(step):
    #getTPSLfrom_telegram()
    try:
        x = threading.Thread(target=check_positions_new(symbol))
        x.start()
        y = threading.Thread(target=check_positions_ready(symbol))
        y.start()
        z = threading.Thread(target=check_one_stop(symbol))
        z.start()
    except :
        print('\n\nSomething went wrong. Continuing...')

In [32]:
telegram_delay=12
bot_token='5454990445:AAGzfeY80oeI-1su0MEIRVuvTzBYIVlZIXU'
chat_id='92428939'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(strr)
    rs=response.json()
    exit=0

    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']        
        if(time.time()-datet)<telegram_delay:
            if 'exit' in textt:
                exit=1
                telegram_bot_sendtext('Right now! Sir')
            if 'profit' in textt:
                trades = client.futures_account_trades(symbol=symbol)
                dataset=pd.DataFrame(trades).iloc[-50:]
                bnbdata=dataset[dataset['commissionAsset']=='BNB']
                busddata=dataset[dataset['commissionAsset']=='BUSD']
                bnbcom=bnbdata['commission'].astype(float).sum()
                busdcom=busddata['commission'].astype(float).sum()
                com=(bnbcom*270+busdcom)*-1
                real=dataset['realizedPnl'].astype(float).sum()
                profit=com+real
                telegram_bot_sendtext('Profit = ' + str(profit))

    return(exit)

In [33]:
trades = client.futures_account_trades(symbol=symbol)
dataset=pd.DataFrame(trades).iloc[-30:]
bnbdata=dataset[dataset['commissionAsset']=='BNB']
busddata=dataset[dataset['commissionAsset']==asset]
bnbcom=bnbdata['commission'].astype(float).sum()
busdcom=busddata['commission'].astype(float).sum()
com=(bnbcom*270+busdcom)*-1
real=dataset['realizedPnl'].astype(float).sum()
profit=com+real
profit

-0.31278711000000003

In [34]:
def telegram_bot_sendtext(bot_message):
    bot_token2 = bot_token
    bot_chatID = chat_id
    send_text = 'https://api.telegram.org/bot' + bot_token2 + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

In [35]:
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)

In [36]:
def start_stop():
    starttime=time.time()
    counterr=1
    timeout = time.time() + 60*60*2  # 60 seconds times 60 meaning the script will run for 12 hr
        
    while time.time() <= timeout:
        try:

            print("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            main(counterr)
            counterr=counterr+1
            # stop=getTPSLfrom_telegram()
            # if stop==1:
            #     break
            if counterr>3:
                # break
                counterr=1
            time.sleep(7 - ((time.time() - starttime) % 7.0)) # 1 minute interval between each new execution
        except KeyboardInterrupt:
            print('\n\KeyboardInterrupt. Stopping.')
        

In [37]:
start_stop()


script continue running at 2022-11-12 18:30:55
0.0011923613170231428
Near average price
script continue running at 2022-11-12 18:31:02
0.0009176774841569835
Near average price
script continue running at 2022-11-12 18:31:09
8.17045267779033e-05
Near average price
script continue running at 2022-11-12 18:31:16
0.00021577566145194224
Near average price
script continue running at 2022-11-12 18:31:23
0.0002394352734532116
Near average price
script continue running at 2022-11-12 18:31:30
0.0004918044681337766
Near average price
script continue running at 2022-11-12 18:31:37
0.0004839179308000201
Near average price
script continue running at 2022-11-12 18:31:44
0.0007047409761453801
Near average price
script continue running at 2022-11-12 18:31:51
0.0004918044681337766
Near average price
script continue running at 2022-11-12 18:31:58
0.000381392945461007
Near average price
script continue running at 2022-11-12 18:32:05
0.00029449960856979784
Near average price
script continue running at 2022-